```python
import os
import random
from Crypto.Util.strxor import strxor
from math import gcd

with open('secret.txt', 'rb') as f:
    plaintext = f.read()
    keylength = len(plaintext)
    while gcd(keylength, len(plaintext)) > 1:
        keylength = random.randint(10, 100)
    key = os.urandom(keylength)
    key = key * len(plaintext)
    plaintext = plaintext * keylength
    ciphertext = strxor(plaintext, key)
    with open('out.txt', 'w') as g:
        g.write(ciphertext.hex())
```

Bài này sẽ random một key có độ dài là số nguyên tố cùng nhau với độ dài của plaintext. Đặt $l$ là độ dài plaintext và $n$ là độ dài key. Chương trình tạo key mới bằng cách lặp lại key cũ $l$ lần, tương tự plaintext mới sẽ là plaintext cũ lặp lại $n$ lần. Chú ý rằng $\gcd(l, n) = 1$, điều này rất quan trọng giải bài này.

Khi factor độ dài của ciphertext thì mình thấy có hai khả năng xảy ra của độ dài key là 32 hoặc 79. Mình giải với $n = 79$ (sai thì quay lại làm 32 :v).

Như một thói quen, để xử lý các bài toán với số lớn mình thường xem xét những trường hợp số nhỏ để xem mối liên hệ giữa chúng. 

Giả sử $l = 5$ và $n =3$. Đặt key là $\mathbf{k} = (k_0, k_1, k_2)$ và $\mathbf{P} = (p_0, p_1, p_2, p_3, p_4)$.

Khi đó ciphertext sẽ được ghép cặp XOR như sau

$$\begin{pmatrix} k_0 & k_1 & k_2 & k_0 & k_1 & k_2 & k_0 & k_1 & k_2 & k_0 & k_1 & k_2 & k_0 & k_1 & k_2 \\ p_0 & p_1 & p_2 & p_3 & p_4 & p_0 & p_1 & p_2 & p_3 & p_4 & p_0 & p_1 & p_2 & p_3 & p_4 \\ c_0 & c_1 & c_2 & c_3 & c_4 & c_5 & c_6 & c_7 & c_8 & c_9 & c_{10} & c_{11} & c_{12} & c_{13} & c_{14}\end{pmatrix}$$

Nhìn vào $k_0$, mình thấy rằng $k_0$ tác động lần lượt lên $p_0$, $p_3$, $p_1$, $p_4$ và $p_2$, tương ứng với các ciphertext $c_0$, $c_3$, $c_6$, $c_9$ và $c_{12}$. Như vậy mình chỉ cần *sắp xếp* lại $p_0$, $p_3$, ... về đúng vị trí của nó là được.

Vì $\gcd(l, n) = 1$ nên mình nhớ tới một tính chất mà chúng ta hay dùng để chứng minh định lý Wilson hoặc Euler là nếu $\{ g_1, g_2, \ldots, g_{\phi(n)} \}$ là hệ thặng dư thu gọn modulo $n$ và $a$ là số sao cho $\gcd(a, n) = 1$ thì tập $\{ a g_1 \bmod n, a g_2 \bmod n, \ldots, a g_{\phi(n)} \bmod n \}$ cũng là hệ thặng dư thu gọn modulo $n$. Nói cách khác hai tập hợp $\{ g_1, g_2, \ldots, g_{\phi(n)} \}$ và $\{ a g_1 \bmod n, a g_2 \bmod n, \ldots, a g_{\phi(n)} \bmod n \}$ là hoán vị của nhau.

Mình thử như sau:

- Với $i = 0$ thì $0 \cdot 3 = 0 \bmod 5$
- Với $i = 1$ thì $1 \cdot 3 = 3 \bmod 5$
- Với $i = 2$ thì $2 \cdot 3 = 1 \bmod 5$
- Với $i = 3$ thì $3 \cdot 3 = 4 \bmod 5$
- Với $i = 4$ thì $4 \cdot 3 = 2 \bmod 5$

Nhìn chuỗi $(0, 3, 1, 4, 2)$ quen quen. Đó chính là $p_0$, $p_3$, $p_1$, $p_4$, $p_2$ ở trên.

Như vậy mình có công thức tổng quát là $p_{i \cdot 3 \bmod 5} = c_{i \cdot 3}$. Hay tổng quát hơn $p_{i \cdot n \bmod l} = c_{i \cdot n}$ với $i = 0, 1, \ldots, l-1$.

Sau đó mình chỉ cần bruteforce 256 trường hợp $k_0$ nữa. Ở đây mình thấy với $k_0 = 165$ thì có chuỗi `PNG` (?).

In [3]:
with open("out.txt") as f:
    ct = f.read()

ctx = bytes.fromhex(ct)

#n = 32
n = 79
l = len(ctx) // n

bb = [0] * l

for i in range(l):
    bb[(i * n) % l] = ctx[i * n]


for key in range(256):
    flag = [key ^ b for b in bb]
    print(key, bytes(flag[:24]))

0 b'G%\x15\xf5\xeb\xe2\xaf\xbf\xafJ\x1a\x18J\x1a\x18J\x1a\x18\xaf\xec\xed\xe1\xf7J'
1 b'F$\x14\xf4\xea\xe3\xae\xbe\xaeK\x1b\x19K\x1b\x19K\x1b\x19\xae\xed\xec\xe0\xf6K'
2 b"E'\x17\xf7\xe9\xe0\xad\xbd\xadH\x18\x1aH\x18\x1aH\x18\x1a\xad\xee\xef\xe3\xf5H"
3 b'D&\x16\xf6\xe8\xe1\xac\xbc\xacI\x19\x1bI\x19\x1bI\x19\x1b\xac\xef\xee\xe2\xf4I'
4 b'C!\x11\xf1\xef\xe6\xab\xbb\xabN\x1e\x1cN\x1e\x1cN\x1e\x1c\xab\xe8\xe9\xe5\xf3N'
5 b'B \x10\xf0\xee\xe7\xaa\xba\xaaO\x1f\x1dO\x1f\x1dO\x1f\x1d\xaa\xe9\xe8\xe4\xf2O'
6 b'A#\x13\xf3\xed\xe4\xa9\xb9\xa9L\x1c\x1eL\x1c\x1eL\x1c\x1e\xa9\xea\xeb\xe7\xf1L'
7 b'@"\x12\xf2\xec\xe5\xa8\xb8\xa8M\x1d\x1fM\x1d\x1fM\x1d\x1f\xa8\xeb\xea\xe6\xf0M'
8 b'O-\x1d\xfd\xe3\xea\xa7\xb7\xa7B\x12\x10B\x12\x10B\x12\x10\xa7\xe4\xe5\xe9\xffB'
9 b'N,\x1c\xfc\xe2\xeb\xa6\xb6\xa6C\x13\x11C\x13\x11C\x13\x11\xa6\xe5\xe4\xe8\xfeC'
10 b'M/\x1f\xff\xe1\xe8\xa5\xb5\xa5@\x10\x12@\x10\x12@\x10\x12\xa5\xe6\xe7\xeb\xfd@'
11 b'L.\x1e\xfe\xe0\xe9\xa4\xb4\xa4A\x11\x13A\x11\x13A\x11\x13\xa4\xe7\xe6\

In [4]:
flag = bytes(165 ^ b for b in bb)[0x1b2:0x1f8]
print(flag)

b'amateursCTF{M4yb3_H4v3_b3tt3r_0tP_3ncrYpt10n_n3X7_t1m3_l0L!!_a585b81b}'


Okay vậy là xong bài :))))